In [94]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import GridSearchCV

In [95]:
train_data = pd.read_csv('/content/Trainlast1_dataset.csv')
test_data = pd.read_csv('/content/Testlast1_dataset.csv')

In [97]:
# Separate features and target for training
X_train = train_data.drop('is_fraud', axis=1)
y_train = train_data['is_fraud']

In [99]:
# Separate features and target for testing
X_test = test_data.drop('is_fraud', axis=1)
y_test = test_data['is_fraud']

In [100]:
# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [102]:

# Apply SMOTE for oversampling
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)


In [112]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import GlorotUniform

# Create a function to build the model
def create_model(optimizer=Adam(learning_rate=0.001), dropout_rate=0.3, units_1=512, units_2=256):
    model = Sequential([
        Dense(units_1, input_dim=X_train_scaled.shape[1], kernel_initializer=GlorotUniform()),
        Activation('relu'),
        BatchNormalization(),
        Dropout(dropout_rate),

        Dense(units_2, kernel_regularizer=l2(0.001), kernel_initializer=GlorotUniform()),
        Activation('relu'),
        BatchNormalization(),
        Dropout(dropout_rate),

        Dense(units_2, kernel_regularizer=l2(0.001), kernel_initializer=GlorotUniform()),
        Activation('relu'),
        BatchNormalization(),
        Dropout(dropout_rate),

        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [113]:

# Initialize KerasClassifier
keras_classifier = KerasClassifier(build_fn=create_model, verbose=0)

<ipython-input-113-9ef6d4d851ee>:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_classifier = KerasClassifier(build_fn=create_model, verbose=0)


In [115]:
# Define hyperparameters to tune
param_grid = {
    'optimizer': ['adam', 'sgd'],
    'dropout_rate': [0.2, 0.3],
    'units_1': [128, 256],  # Increase units for more capacity
    'units_2': [64, 128]    # Increase units for more capacity
}

In [116]:

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=keras_classifier, param_grid=param_grid, scoring='accuracy', cv=5)



In [117]:
# Fit GridSearchCV on data
grid_search_result = grid_search.fit(X_train_resampled, y_train_resampled)



94/94 [==============================] - 0s 2ms/step


In [118]:

# Get best parameters and best score
best_params = grid_search_result.best_params_
best_score = grid_search_result.best_score_

print(f'Best Parameters: {best_params}')
print(f'Best Score: {best_score}')


Best Parameters: {'dropout_rate': 0.2, 'optimizer': 'adam', 'units_1': 256, 'units_2': 128}
Best Score: 0.8299897970117824


In [119]:
# Use the best model to make predictions
best_model = grid_search_result.best_estimator_
y_pred_probs = best_model.predict(X_test_scaled)
y_pred = (y_pred_probs > 0.5).astype(int)


135/135 [==============================] - 0s 1ms/step


In [121]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_probs)

print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-score: {f1:.2f}')
print(f'ROC-AUC: {roc_auc:.2f}')



Accuracy: 0.87
Precision: 0.97
Recall: 0.77
F1-score: 0.86
ROC-AUC: 0.87
